In [1]:
import pandas as pd
user = pd.read_csv('Jdata/JData_User.csv',encoding='gbk')


In [2]:
user.head()

user_id     age  sex  user_lv_cd user_reg_tm
0   200001   56岁以上  2.0           5  2016-01-26
1   200002      -1  0.0           1  2016-01-26
2   200003  36-45岁  1.0           4  2016-01-26
3   200004      -1  2.0           1  2016-01-26
4   200005  16-25岁  0.0           4  2016-01-26

In [3]:
month2=pd.read_csv('Jdata/JData_Action_201602.csv')
month3=pd.read_csv('Jdata/JData_Action_201603.csv')
month4=pd.read_csv('Jdata/JData_Action_201604.csv')
comment=pd.read_csv('Jdata/JData_Comment.csv')
product=pd.read_csv('Jdata/JData_Product.csv')

In [4]:
month2.head()

user_id  sku_id                 time  model_id  type  cate  brand
0  266079.0  138778  2016-01-31 23:59:02       NaN     1     8    403
1  266079.0  138778  2016-01-31 23:59:03       0.0     6     8    403
2  200719.0   61226  2016-01-31 23:59:07       NaN     1     8     30
3  200719.0   61226  2016-01-31 23:59:08       0.0     6     8     30
4  263587.0   72348  2016-01-31 23:59:08       NaN     1     5    159

In [6]:
product.head()

sku_id  a1  a2  a3  cate  brand
0      10   3   1   1     8    489
1  100002   3   2   2     8    489
2  100003   1  -1  -1     8     30
3  100006   1   2   1     8    545
4   10001  -1   1   2     8    244

In [5]:
comment.head()

dt  sku_id  comment_num  has_bad_comment  bad_comment_rate
0  2016-02-01    1000            3                1            0.0417
1  2016-02-01   10000            2                0            0.0000
2  2016-02-01  100011            4                1            0.0376
3  2016-02-01  100018            3                0            0.0000
4  2016-02-01  100020            3                0            0.0000

## frist we check whether user data is consistent with action data

In [7]:
sku = user.ix[:,'user_id'].to_frame()

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  """Entry point for launching an IPython kernel.


In [10]:
print ('Is action of Feb. from User file? ')
len(month2) == len(pd.merge(sku,month2))


Is action of Feb. from User file? 


True

In [11]:
print ('Is action of Feb. from User file? ')
len(month3) == len(pd.merge(sku,month3))


Is action of Feb. from User file? 


True

In [12]:
print ('Is action of Feb. from User file? ')
len(month4) == len(pd.merge(sku,month4))


Is action of Feb. from User file? 


True

## remove rows that user_reg_tm after 2016/4/15

In [16]:
user['user_reg_tm']=pd.to_datetime(user['user_reg_tm'])
user.ix[user.user_reg_tm  > '2016-4-15']

user_id     age  sex  user_lv_cd user_reg_tm
8545    208546  16-25岁  0.0           2  2016-04-29
9394    209395  16-25岁  1.0           2  2016-05-11
10362   210363   56岁以上  2.0           2  2016-05-24
10367   210368      -1  2.0           1  2016-05-24
11019   211020  36-45岁  2.0           3  2016-06-06
12014   212015  36-45岁  2.0           2  2016-07-05
13850   213851  26-35岁  2.0           3  2016-09-11
14542   214543      -1  2.0           1  2016-10-05
16746   216747  16-25岁  2.0           1  2016-11-25

In [19]:
month4['time'] = pd.to_datetime(month4['time'])
month4.ix[month4.time >= '2016-4-16']

Empty DataFrame
Columns: [user_id, sku_id, time, model_id, type, cate, brand]
Index: []

### however these are normal users,the date issue is due to system error, now we change the type of user_id into int

In [20]:
month2['user_id'] = month2['user_id'].apply(lambda x:int(x))
month3['user_id'] = month3['user_id'].apply(lambda x:int(x))
month4['user_id'] = month4['user_id'].apply(lambda x:int(x))


In [23]:
def tranAge(x):
    if x == u'15岁以下':
        x='1'
    elif x==u'16-25岁':
        x='2'
    elif x==u'26-35岁':
        x='3'
    elif x==u'36-45岁':
        x='4'
    elif x==u'46-55岁':
        x='5'
    elif x==u'56岁以上':
        x='6'
    return x
user['age']=user['age'].apply(tranAge)
print(user.groupby(user['age']).count())

     user_id    sex  user_lv_cd  user_reg_tm
age                                         
-1     14412  14412       14412        14412
1          7      7           7            7
2       8797   8797        8797         8797
3      46570  46570       46570        46570
4      30336  30336       30336        30336
5       3325   3325        3325         3325
6       1871   1871        1871         1871


### create user table

In [22]:
#定义文件名
ACTION_201602_FILE = "Jdata/JData_Action_201602.csv"
ACTION_201603_FILE = "Jdata/JData_Action_201603.csv"
ACTION_201604_FILE = "Jdata/JData_Action_201604.csv"
COMMENT_FILE = "Jdata/JData_Comment.csv"
PRODUCT_FILE = "Jdata/JData_Product.csv"
USER_FILE = "Jdata/JData_User.csv"
USER_TABLE_FILE = "Jdata/User_table.csv"
ITEM_TABLE_FILE = "Jdata/Item_table.csv"

In [28]:
import numpy as np
from collections import Counter

In [26]:
# 功能函数: 对每一个user分组的数据进行统计
def add_type_count(group):
    behavior_type = group.type.astype(int)
    # 用户行为类别
    type_cnt = Counter(behavior_type)
    # 1: 浏览 2: 加购 3: 删除
    # 4: 购买 5: 收藏 6: 点击
    group['browse_num'] = type_cnt[1]
    group['addcart_num'] = type_cnt[2]
    group['delcart_num'] = type_cnt[3]
    group['buy_num'] = type_cnt[4]
    group['favor_num'] = type_cnt[5]
    group['click_num'] = type_cnt[6]

    return group[['user_id', 'browse_num', 'addcart_num',
                  'delcart_num', 'buy_num', 'favor_num',
                  'click_num']]

In [27]:
#对action数据进行统计
#根据自己调节chunk_size大小
def get_from_action_data(fname, chunk_size=100000):
    reader = pd.read_csv(fname, header=0, iterator=True)
    chunks = []
    loop = True
    while loop:
        try:
            # 只读取user_id和type两个字段
            chunk = reader.get_chunk(chunk_size)[["user_id", "type"]]
            chunks.append(chunk)
        except StopIteration:
            loop = False
            print("Iteration is stopped")
    # 将块拼接为pandas dataframe格式
    df_ac = pd.concat(chunks, ignore_index=True)
    # 按user_id分组，对每一组进行统计，as_index 表示无索引形式返回数据
    df_ac = df_ac.groupby(['user_id'], as_index=False).apply(add_type_count)
    # 将重复的行丢弃
    df_ac = df_ac.drop_duplicates('user_id')

    return df_ac

In [28]:
# 将各个action数据的统计量进行聚合
def merge_action_data():
    df_ac = []
    df_ac.append(get_from_action_data(fname=ACTION_201602_FILE))
    df_ac.append(get_from_action_data(fname=ACTION_201603_FILE))
    df_ac.append(get_from_action_data(fname=ACTION_201604_FILE))

    df_ac = pd.concat(df_ac, ignore_index=True)
    # 用户在不同action表中统计量求和
    df_ac = df_ac.groupby(['user_id'], as_index=False).sum()
    #　构造转化率字段
    df_ac['buy_addcart_ratio'] = df_ac['buy_num'] / df_ac['addcart_num']
    df_ac['buy_browse_ratio'] = df_ac['buy_num'] / df_ac['browse_num']
    df_ac['buy_click_ratio'] = df_ac['buy_num'] / df_ac['click_num']
    df_ac['buy_favor_ratio'] = df_ac['buy_num'] / df_ac['favor_num']
    
    # 将大于１的转化率字段置为１(100%)
    df_ac.ix[df_ac['buy_addcart_ratio'] > 1., 'buy_addcart_ratio'] = 1.
    df_ac.ix[df_ac['buy_browse_ratio'] > 1., 'buy_browse_ratio'] = 1.
    df_ac.ix[df_ac['buy_click_ratio'] > 1., 'buy_click_ratio'] = 1.
    df_ac.ix[df_ac['buy_favor_ratio'] > 1., 'buy_favor_ratio'] = 1.

    return df_ac

In [31]:
#　从FJData_User表中抽取需要的字段
def get_from_jdata_user():
    df_usr = pd.read_csv(USER_FILE, header=0,encoding='gbk')
    df_usr = df_usr[["user_id", "age", "sex", "user_lv_cd"]]
    return df_usr

In [32]:
user_base = get_from_jdata_user()
user_behavior = merge_action_data()

# 连接成一张表，类似于SQL的左连接(left join)
user_behavior = pd.merge(user_base, user_behavior, on=['user_id'], how='left')
# 保存为user_table.csv
user_behavior.to_csv(USER_TABLE_FILE, index=False)

Iteration is stopped
Iteration is stopped
Iteration is stopped


## clean user data

- show overall data

In [37]:
df_user = pd.read_csv('Jdata/User_table.csv',header=0)
pd.options.display.float_format = '{:,.3f}'.format  #输出格式设置，保留三位小数
df_user.describe()

user_id         sex  user_lv_cd  browse_num  addcart_num  \
count 105,321.000 105,318.000 105,321.000 105,180.000  105,180.000   
mean  252,661.000       1.113       3.850     180.466        5.471   
std    30,403.698       0.956       1.072     273.437       10.618   
min   200,001.000       0.000       1.000       0.000        0.000   
25%   226,331.000       0.000       3.000      40.000        0.000   
50%   252,661.000       2.000       4.000      94.000        2.000   
75%   278,991.000       2.000       5.000     212.000        6.000   
max   305,321.000       2.000       5.000   7,605.000      369.000   

       delcart_num     buy_num   favor_num   click_num  buy_addcart_ratio  \
count  105,180.000 105,180.000 105,180.000 105,180.000         72,129.000   
mean         2.434       0.459       1.045     291.222              0.147   
std          5.600       1.048       3.442     460.031              0.270   
min          0.000       0.000       0.000       0.000              0.000   
25%          0.000       0.000       0.000      59.000              0.000   
50%          0.000       0.000       0.000     148.000              0.000   
75%          3.000       1.000       0.000     342.000              0.167   
max        231.000      50.000      99.000  15,302.000              1.000   

       buy_browse_ratio  buy_click_ratio  buy_favor_ratio  
count       105,172.000      103,197.000       45,986.000  
mean              0.005            0.009            0.552  
std               0.022            0.074            0.473  
min               0.000            0.000            0.000  
25%               0.000            0.000            0.000  
50%               0.000            0.000            1.000  
75%               0.002            0.001            1.000  
max               1.000            1.000            1.000

In [ ]:
df_user[df_user['age'].isnull()]

- delete rows with no age and sex

In [38]:
delete_list = df_user[df_user['age'].isnull()].index
df_user.drop(delete_list,axis=0,inplace=True)

- delete rows with no actions

In [40]:
#删除无交互记录的用户
df_naction = df_user[(df_user['browse_num'].isnull()) & (df_user['addcart_num'].isnull()) & (df_user['delcart_num'].isnull()) & (df_user['buy_num'].isnull()) & (df_user['favor_num'].isnull()) & (df_user['click_num'].isnull())]
df_user.drop(df_naction.index,axis=0,inplace=True)
len(df_user)

105177

In [42]:
#统计无购买记录的用户
df_bzero = df_user[df_user['buy_num']==0]
#输出购买数为0的总记录数
len(df_bzero)

75694

In [44]:
#删除无购买记录的用户
df_user = df_user[df_user['buy_num']!=0]

In [45]:
df_user.describe()

user_id        sex  user_lv_cd  browse_num  addcart_num  \
count  29,483.000 29,483.000  29,483.000  29,483.000   29,483.000   
mean  250,746.445      1.025       4.272     302.488       10.525   
std    29,979.676      0.959       0.808     391.535       14.301   
min   200,001.000      0.000       2.000       1.000        0.000   
25%   225,058.500      0.000       4.000      76.000        3.000   
50%   249,144.000      1.000       4.000     178.000        6.000   
75%   276,252.500      2.000       5.000     381.000       13.000   
max   305,318.000      2.000       5.000   7,605.000      288.000   

       delcart_num    buy_num  favor_num  click_num  buy_addcart_ratio  \
count   29,483.000 29,483.000 29,483.000 29,483.000         29,483.000   
mean         4.673      1.637      1.677    486.653              0.360   
std          7.568      1.412      4.584    658.671              0.320   
min          0.000      1.000      0.000      0.000              0.004   
25%          0.000      1.000      0.000    116.000              0.118   
50%          2.000      1.000      0.000    282.000              0.250   
75%          6.000      2.000      1.000    604.000              0.500   
max        178.000     50.000     96.000 15,302.000              1.000   

       buy_browse_ratio  buy_click_ratio  buy_favor_ratio  
count        29,483.000       29,483.000       29,483.000  
mean              0.018            0.030            0.862  
std               0.038            0.136            0.287  
min               0.000            0.000            0.010  
25%               0.004            0.002            1.000  
50%               0.008            0.005            1.000  
75%               0.018            0.012            1.000  
max               1.000            1.000            1.000

- delete lazy users

In [49]:
bindex = df_user[df_user['buy_browse_ratio']<0.0005].index
df_user.drop(bindex,axis=0,inplace=True)

In [50]:
cindex = df_user[df_user['buy_click_ratio']<0.0005].index
df_user.drop(cindex,axis=0,inplace=True)
df_user.describe()

user_id        sex  user_lv_cd  browse_num  addcart_num  \
count  29,070.000 29,070.000  29,070.000  29,070.000   29,070.000   
mean  250,767.099      1.028       4.268     280.260       10.145   
std    29,998.870      0.959       0.809     325.129       13.443   
min   200,001.000      0.000       2.000       1.000        0.000   
25%   225,036.000      0.000       4.000      75.000        3.000   
50%   249,200.500      1.000       4.000     174.000        6.000   
75%   276,284.000      2.000       5.000     366.000       13.000   
max   305,318.000      2.000       5.000   5,007.000      288.000   

       delcart_num    buy_num  favor_num  click_num  buy_addcart_ratio  \
count   29,070.000 29,070.000 29,070.000 29,070.000         29,070.000   
mean         4.457      1.644      1.589    447.113              0.364   
std          6.998      1.420      4.294    530.994              0.320   
min          0.000      1.000      0.000      0.000              0.004   
25%          0.000      1.000      0.000    114.000              0.125   
50%          2.000      1.000      0.000    275.000              0.250   
75%          6.000      2.000      1.000    585.000              0.500   
max        158.000     50.000     69.000  8,156.000              1.000   

       buy_browse_ratio  buy_click_ratio  buy_favor_ratio  
count        29,070.000       29,070.000       29,070.000  
mean              0.019            0.031            0.866  
std               0.038            0.137            0.282  
min               0.001            0.001            0.018  
25%               0.004            0.002            1.000  
50%               0.008            0.005            1.000  
75%               0.018            0.012            1.000  
max               1.000            1.000            1.000

## save the final user table

In [56]:
df_user.to_csv("Jdata/JData_FUser.csv",index=None)

In [57]:
final_user=pd.read_csv('Jdata/JData_FUser.csv',header=0)
final_user.head()


user_id     age   sex  user_lv_cd  browse_num  addcart_num  delcart_num  \
0   200001   56岁以上 2.000           5     212.000       22.000       13.000   
1   200005  16-25岁 0.000           4     106.000        2.000        3.000   
2   200014  26-35岁 2.000           4     468.000       29.000       14.000   
3   200015  26-35岁 1.000           3   1,261.000       15.000        4.000   
4   200017  36-45岁 2.000           4     114.000        0.000        0.000   

   buy_num  favor_num  click_num  buy_addcart_ratio  buy_browse_ratio  \
0    1.000      0.000    414.000              0.045             0.005   
1    1.000      2.000    161.000              0.500             0.009   
2    2.000      0.000    645.000              0.069             0.004   
3    1.000      3.000  1,979.000              0.067             0.001   
4    4.000      0.000    187.000              1.000             0.035   

   buy_click_ratio  buy_favor_ratio  
0            0.002            1.000  
1            0.006            0.500  
2            0.003            1.000  
3            0.001            0.333  
4            0.021            1.000

## create item table

In [16]:
def get_from_jdata_product():
    df_item = pd.read_csv(PRODUCT_FILE, header=0)
    return df_item

In [29]:
import numpy as np
from collections import Counter

In [30]:
# apply type count
def add_type_count(group):
    behavior_type = group.type.astype(int)
    type_cnt = Counter(behavior_type)

    group['browse_num'] = type_cnt[1]
    group['addcart_num'] = type_cnt[2]
    group['delcart_num'] = type_cnt[3]
    group['buy_num'] = type_cnt[4]
    group['favor_num'] = type_cnt[5]
    group['click_num'] = type_cnt[6]

    return group[['sku_id', 'browse_num', 'addcart_num',
                  'delcart_num', 'buy_num', 'favor_num',
                  'click_num']]

In [31]:
def get_from_action_data(fname, chunk_size=100000):
    reader = pd.read_csv(fname, header=0, iterator=True)
    chunks = []
    loop = True
    while loop:
        try:
            chunk = reader.get_chunk(chunk_size)[["sku_id", "type"]]
            chunks.append(chunk)
        except StopIteration:
            loop = False
            print("Iteration is stopped")

    df_ac = pd.concat(chunks, ignore_index=True)

    df_ac = df_ac.groupby(['sku_id'], as_index=False).apply(add_type_count)
    # Select unique row
    df_ac = df_ac.drop_duplicates('sku_id')

    return df_ac

In [32]:
def get_from_jdata_comment():
    df_cmt = pd.read_csv(COMMENT_FILE, header=0)
    df_cmt['dt'] = pd.to_datetime(df_cmt['dt'])
    # find latest comment index
    idx = df_cmt.groupby(['sku_id'])['dt'].transform(max) == df_cmt['dt']
    df_cmt = df_cmt[idx]

    return df_cmt[['sku_id', 'comment_num',
                   'has_bad_comment', 'bad_comment_rate']]

In [33]:
def merge_action_data():
    df_ac = []
    df_ac.append(get_from_action_data(fname=ACTION_201602_FILE))
    df_ac.append(get_from_action_data(fname=ACTION_201603_FILE))
    df_ac.append(get_from_action_data(fname=ACTION_201604_FILE))

    df_ac = pd.concat(df_ac, ignore_index=True)
    df_ac = df_ac.groupby(['sku_id'], as_index=False).sum()

    df_ac['buy_addcart_ratio'] = df_ac['buy_num'] / df_ac['addcart_num']
    df_ac['buy_browse_ratio'] = df_ac['buy_num'] / df_ac['browse_num']
    df_ac['buy_click_ratio'] = df_ac['buy_num'] / df_ac['click_num']
    df_ac['buy_favor_ratio'] = df_ac['buy_num'] / df_ac['favor_num']

    df_ac.ix[df_ac['buy_addcart_ratio'] > 1., 'buy_addcart_ratio'] = 1.
    df_ac.ix[df_ac['buy_browse_ratio'] > 1., 'buy_browse_ratio'] = 1.
    df_ac.ix[df_ac['buy_click_ratio'] > 1., 'buy_click_ratio'] = 1.
    df_ac.ix[df_ac['buy_favor_ratio'] > 1., 'buy_favor_ratio'] = 1.

    return df_ac

In [34]:
item_base = get_from_jdata_product()
item_behavior = merge_action_data()
item_comment = get_from_jdata_comment()

Iteration is stopped
Iteration is stopped
Iteration is stopped


/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:15: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  from ipykernel import kernelapp as app
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:16: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  app.launch_new_instance()
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:17: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.

In [35]:
# SQL: left join
item_behavior = pd.merge(
    item_base, item_behavior, on=['sku_id'], how='left')
item_behavior = pd.merge(
    item_behavior, item_comment, on=['sku_id'], how='left')

item_behavior.to_csv(ITEM_TABLE_FILE, index=False)

## show final item table

In [37]:
product = pd.read_csv('Jdata/Item_table.csv',header=0)
pd.options.display.float_format = '{:,.3f}'.format  #输出格式设置，保留三位小数
product.describe()

sku_id         a1         a2         a3       cate      brand  \
count  24,187.000 24,187.000 24,187.000 24,187.000 24,187.000 24,187.000   
mean   85,398.737      2.177      0.939      1.180      8.000    435.864   
std    49,238.799      1.176      0.970      1.046      0.000    225.749   
min         6.000     -1.000     -1.000     -1.000      8.000      3.000   
25%    42,476.000      1.000      1.000      1.000      8.000    214.000   
50%    85,616.000      3.000      1.000      1.000      8.000    489.000   
75%   127,774.000      3.000      2.000      2.000      8.000    571.000   
max   171,224.000      3.000      2.000      2.000      8.000    922.000   

       browse_num  addcart_num  delcart_num   buy_num  favor_num   click_num  \
count   3,938.000    3,938.000    3,938.000 3,938.000  3,938.000   3,938.000   
mean    1,723.711       54.212       21.284     3.373     10.655   2,790.132   
std     7,957.661      285.723      106.100    21.695     49.956  12,647.534   
min         0.000        0.000        0.000     0.000      0.000       0.000   
25%        19.000        0.000        0.000     0.000      0.000      30.000   
50%       148.000        1.000        1.000     0.000      1.000     246.500   
75%       655.750       12.000        5.000     0.000      4.000   1,071.750   
max   194,920.000    6,296.000    2,258.000   691.000  1,205.000 312,005.000   

       buy_addcart_ratio  buy_browse_ratio  buy_click_ratio  buy_favor_ratio  \
count          2,225.000         3,909.000        3,692.000        2,016.000   
mean               0.036             0.001            0.000            0.162   
std                0.093             0.002            0.001            0.264   
min                0.000             0.000            0.000            0.000   
25%                0.000             0.000            0.000            0.000   
50%                0.000             0.000            0.000            0.000   
75%                0.048             0.000            0.000            0.250   
max                1.000             0.114            0.063            1.000   

       comment_num  has_bad_comment  bad_comment_rate  
count    6,830.000        6,830.000         6,830.000  
mean         2.732            0.486             0.044  
std          1.037            0.500             0.110  
min          1.000            0.000             0.000  
25%          2.000            0.000             0.000  
50%          3.000            0.000             0.000  
75%          4.000            1.000             0.044  
max          4.000            1.000             1.000